In [2]:
# load and auto-reload modules
%load_ext autoreload
%autoreload 2

In [3]:
# para usar los modulos de src
import sys
from pathlib import Path
notebook_path = Path().resolve()
project_root = notebook_path.parent
sys.path.append(str(project_root))

# carga las variables de entorno
from dotenv import load_dotenv
load_dotenv()

True

# Preparar datos para LLM

In [4]:
# importar modulos
import pandas as pd
import numpy as np
from pathlib import Path
import os
from src.modules.paths_reference import ROOT_PATH

## Preprocesar los datos de articulos

In [7]:
# read all plain files in the data/articulos
articulos_path = ROOT_PATH / 'data' / 'articulos'
articulos = []
for file in articulos_path.iterdir():
    with open(file, 'r', encoding='utf-8') as f:
        articulos.append(f.read())

In [27]:
# create a funtion to extract data from the plain text
import re
def extract_data(text):
    # extract title, search the word title: and extract the text after it
    title = re.search(r'(?<=Título: ).+(?=\n)', text).group()
    # extract link, search the word Enlace: and extract the text after it
    link = re.search(r'(?<=Enlace: ).+(?=\n)', text).group()
    # extract body, search the word Contenido: consider the line breaks
    body = re.search(r'(?<=Contenido:)[\s\S]*?(?=\nTe puede interesar:|\Z)', text, flags=re.DOTALL).group()
   
    return title, link, body

In [28]:
# create a dataframe with the data extracted, puth the data in a list of dictionaries
data = []
for articulo in articulos:
    title, link, body = extract_data(articulo)
    data.append({'title': title, 'link': link, 'body': body})

In [30]:
# create a dataframe
df = pd.DataFrame(data)

In [31]:
df.head()

,title,link,body
0,“El año pasado hubo 6.600 denuncias contra pol...,https://ecuadorverifica.org/2023/09/20/el-ano-...,"\nSegún cifras oficiales, apenas 34 denuncias ..."
1,“Más del 50% de los detenidos sólo han acabado...,https://ecuadorverifica.org/2023/09/08/mas-del...,\nLas cifras ofrecidas por la candidata son ce...
2,“Cerca de 332 mujeres que murieron el año ante...,https://ecuadorverifica.org/2023/09/08/cerca-d...,\nLa cifra la han confirmado colectivos de la ...
3,“Encontraron una piscina de tilapia en una de ...,https://ecuadorverifica.org/2023/09/05/encontr...,\nEl hallazgo se dio durante una de las interv...
4,«El 40% del parque automotor (de la Policía) e...,https://ecuadorverifica.org/2023/08/15/el-40-d...,"\nSegún las cifras oficiales, el 58% de los ca..."


In [37]:
(ROOT_PATH / 'data/results' / 'articulos.xlsx').is_file()

True

In [33]:
df.to_excel(ROOT_PATH / 'data/processed' / 'articulos.xlsx', index=False)

In [39]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [62]:
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

class FormatedTextOutputParser(BaseModel):
    summary: str = Field(description="El resultado del texto formateado")

class FormatedQuestionOutputParser(BaseModel):
    summary: str = Field(description="El resultado del texto formateado")
    
def get_boy_formated(llm, text: str) -> str: 
    parser = JsonOutputParser(pydantic_object=FormatedTextOutputParser)
    prompt = PromptTemplate(
    template="""
Reformula el siguiente texto para que fluya de manera coherente y natural, eliminando los saltos de línea innecesarios. Si el texto tiene una calificación asociada, colócala al principio. Adapta el estilo para que suene como una respuesta informativa de un asistente de IA, en lugar de un artículo de investigación. Omite o adapta cualquier referencia a entrevistas. Si se menciona 'Ecuador Verifica', modifica el texto para que se refiera a medios de comunicación o periodistas en general. El texto resultante debe ser una respuesta fluida y cohesiva, como si fuera generada por un LLM en respuesta a una consulta de información.

{text}

{format_instructions}
                """,
                input_variables=["text"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    chain = prompt | llm | parser
    response = chain.invoke({"text": text})
    return response['summary']

def get_question_formated(llm, text: str) -> str: 
    parser = JsonOutputParser(pydantic_object=FormatedQuestionOutputParser)
    prompt = PromptTemplate(
    template="""
Basándote en la siguiente información, genera una pregunta que simule a una persona buscando verificar una declaración hecha por un político en un discurso. La pregunta debe:

Reflejar la esencia de la información proporcionada.
Ser formulada como si fuera de un ciudadano común buscando aclaración sobre el discurso escuchado.
Mencionar específicamente que la declaración proviene de un político si se menciona su nombre incluirlo.
Solicitar explícitamente una verificación de los hechos.
Ser concisa pero incluir los detalles clave de la información dada.

Por favor, genera la pregunta basándote en la siguiente información:

{text}

{format_instructions}
                """,
                input_variables=["text"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    chain = prompt | llm | parser
    response = chain.invoke({"text": text})
    return response['summary']


In [53]:

text_test = df.body[0]
text_formated = get_boy_formated(llm, text_test)
print(text_formated)

{'summary': 'La afirmación de Luisa González, candidata presidencial por la Revolución Ciudadana, sobre que el año pasado se registraron alrededor de 6.600 denuncias contra policías, de las cuales solo el 2% resultó en destitución, ha sido verificada y calificada como falsa. Según cifras oficiales, solo se registraron 34 denuncias en 2022, de las cuales tres policías fueron destituidos. La mayor parte de los sumarios administrativos se concentraron en Quito, con 12 casos, seguidos por Bolívar con 6 y Esmeraldas con 4. Las razones de las destituciones incluyeron la obtención de beneficios personales y el uso de documentos falsos. A pesar de las solicitudes de información, no se obtuvo respuesta del equipo de comunicación de la candidata ni de la Fiscalía General sobre el número de denuncias, ya que esta información debe ser solicitada al departamento de Asuntos Internos de la Policía.'}


In [54]:
text_formated['summary']

'La afirmación de Luisa González, candidata presidencial por la Revolución Ciudadana, sobre que el año pasado se registraron alrededor de 6.600 denuncias contra policías, de las cuales solo el 2% resultó en destitución, ha sido verificada y calificada como falsa. Según cifras oficiales, solo se registraron 34 denuncias en 2022, de las cuales tres policías fueron destituidos. La mayor parte de los sumarios administrativos se concentraron en Quito, con 12 casos, seguidos por Bolívar con 6 y Esmeraldas con 4. Las razones de las destituciones incluyeron la obtención de beneficios personales y el uso de documentos falsos. A pesar de las solicitudes de información, no se obtuvo respuesta del equipo de comunicación de la candidata ni de la Fiscalía General sobre el número de denuncias, ya que esta información debe ser solicitada al departamento de Asuntos Internos de la Policía.'

In [63]:
question = df.title[0]
question_formated = get_question_formated(llm, question)
print(question_formated)

¿Es cierto que Luisa González, la candidata presidencial de Revolución Ciudadana, afirmó que el año pasado hubo 6.600 denuncias contra policías y que solo el 2% fue destituido? Me gustaría verificar esta información.


In [64]:
# create a new dataframe with the formated text and questions
data_formated = []
# iterate over the rows of the original dataframe
for index, row in df.iterrows():
    # get the formated question
    question = row.title
    question_formated = get_question_formated(llm, question)
    # get the formated text
    text_input = row.body
    text_formated = get_boy_formated(llm, text_input)
    # append the data to the list
    data_formated.append({'title': row.title, 'link': row.link, 'question': question_formated,'body': text_formated})

In [66]:
df_formated = pd.DataFrame(data_formated)

In [68]:
df_formated.to_excel(ROOT_PATH / 'data/processed' / 'articulos_formated.xlsx', index=False)

### sumar informacion de afirmaciones de asambleistas

In [6]:
import pandas as pd

df_discurso_politico = pd.read_excel(ROOT_PATH / 'data/processed' / 'articulos con formato.xlsx')
df_asambleistas = pd.read_csv(ROOT_PATH / 'data/processed' / 'afirmaciones_asamblea.csv',encoding='utf-8')

In [7]:
df_fact_check = pd.concat([df_discurso_politico.drop(columns='link'), df_asambleistas], ignore_index=True)

In [11]:
df_fact_check.to_excel(ROOT_PATH / 'data/processed' / 'fact_check.xlsx', index=False)

## GPT

In [12]:
df_fact_check = pd.read_excel(ROOT_PATH / 'data/processed' / 'fact_check.xlsx')

In [13]:
import json

In [14]:
df_fact_check.columns

Index(['title', 'question', 'body'], dtype='object')

In [15]:
# Preparar la lista para los datos formateados
formatted_data = []

# Iterar sobre las filas del DataFrame
for index, row in df_fact_check.iterrows():
    conversation = {
        "messages": [
            {"role": "system", "content": "Eres un asistente de IA que valida informacion de discursos politicos"},
            {"role": "user", "content": row['question']},
            {"role": "assistant", "content": row['body']}
        ]
    }
    formatted_data.append(conversation)



In [17]:
path_to_save = ROOT_PATH / 'data' / 'final' / 'datos_entrenamiento_gpt.json'
# Guardar los datos formateados en un archivo JSON
with open(path_to_save, 'w', encoding='utf-8') as f:
    json.dump(formatted_data, f, ensure_ascii=False, indent=2)

print("Archivo JSON creado con éxito.")

Archivo JSON creado con éxito.


In [20]:
# Cargar el archivo JSON

def json_to_jsonl(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        data = json.load(infile)
        
        # Si el archivo JSON es una lista
        if isinstance(data, list):
            for entry in data:
                outfile.write(json.dumps(entry) + '\n')
        # Si el archivo JSON es un diccionario
        elif isinstance(data, dict):
            outfile.write(json.dumps(data) + '\n')
        else:
            raise ValueError("El archivo JSON debe contener un objeto o una lista en la raíz.")

# Uso del código
input_file = ROOT_PATH / 'data' / 'final' / 'datos_entrenamiento_rag.json'
output_file = ROOT_PATH / 'data' / 'final' / 'datos_entrenamiento_ragL.jsonl'
json_to_jsonl(input_file, output_file)

## LLAMA